In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds 

In [2]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20


In [3]:
mnist_dataset, mnist_info = tfds.load(name ='mnist',with_info=True,as_supervised = True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'] , mnist_dataset['test']

In [5]:
def scale(image,label):
    image = tf.cast(image , tf.float32)
    image /= 255.
    
    return image , label

In [6]:
train_and_validation_data = mnist_train.map(scale)

In [7]:
test_data = mnist_test.map(scale)

In [8]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples , tf.int64)

In [9]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

In [10]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [11]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [12]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

CREATING AND BUILDING THE MODEL

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5, activation='relu',input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [14]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [16]:
model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [ ]:
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping],
    validation_data = validation_data,
    verbose = 2
)

Epoch 1/20
422/422 - 40s - loss: 0.2534 - accuracy: 0.9280 - val_loss: 0.0759 - val_accuracy: 0.9772 - 40s/epoch - 94ms/step
Epoch 2/20
422/422 - 37s - loss: 0.0715 - accuracy: 0.9786 - val_loss: 0.0585 - val_accuracy: 0.9818 - 37s/epoch - 88ms/step
Epoch 3/20
422/422 - 38s - loss: 0.0521 - accuracy: 0.9844 - val_loss: 0.0512 - val_accuracy: 0.9850 - 38s/epoch - 89ms/step
Epoch 4/20
422/422 - 37s - loss: 0.0427 - accuracy: 0.9874 - val_loss: 0.0424 - val_accuracy: 0.9865 - 37s/epoch - 89ms/step
Epoch 5/20
422/422 - 37s - loss: 0.0367 - accuracy: 0.9888 - val_loss: 0.0344 - val_accuracy: 0.9898 - 37s/epoch - 87ms/step
Epoch 6/20


TEST THE MODEL : 

In [ ]:
test_loss , test_accuracy = model.evaluate(test_data)

In [ ]:
print('Test loss : {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss,test_accuracy*100.))

PLOT IMAGES AND RESULTS: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# split the test data into 2 arrays, containing the images and corresponding labels.

for images,labels in test_data.take(1):
    images_test = images.numpy()
    labels_test = labels.numpy()
    
# reshape the images into 28 x 28 form, suitable for matplotlib (orignally dimensions were : 28x28x1)

images_plot = np.reshape(images_test , (10000,28,28))

In [ ]:
i = 15

#plotting image: 
plt.figure(figsize=(2,2))
plt.axis('off')
plt.imshow(images_plot[i-1] , cmap = 'gray', aspect = 'auto')
plt.show()

#print label for image: 
print("Label : {}".format(labels_test[i-1]))

In [ ]:
#obtain model's predictions bts:
predictions = model.predict(images_test[i-1:i])

#convert predictions to probabilities: 
probabilities = tf.nn.softmax(predictions).numpy()
probabilities = probabilities*100

#create bar graph to plot probabilities:
plt.figure(figsize=(12,5))
plt.bar(x=[1,2,3,4,5,6,7,8,9,10], height = probabilities[0],tick_label=['0','1','2','3','4','5','6','7','8','9'])